# Tarea 1 Implementación mlp

## Alumno: Gerardo de Miguel González

### Dataset Circular

In [2]:
#::GMG::Cargo los datos del problema no lineal (nube de puntos ejemplo)
circle <- read.csv('data/circle.csv', header = F)

In [3]:
str(circle)

'data.frame':	100 obs. of  3 variables:
 $ V1: num  -0.9719 -0.553 -0.0949 -0.8096 0.399 ...
 $ V2: num  0.847 -0.577 -0.299 0.857 0.69 ...
 $ V3: int  0 0 1 0 0 0 0 0 0 0 ...


In [4]:
#::GMG::Obtengo los índices de los puntos de una de las clases
ind_c <- which(circle[,3]==0)

In [5]:
par(mar = c(2,5,1,1))
plot(circle[ind_c,1],circle[ind_c,2],
     type='p',
     xlim=c(-1,1),ylim=c(-1,1), 
     xlab = 'x', ylab = 'y',
     col = 'green')
lines(circle[-ind_c,1],circle[-ind_c,2],
      type='p',col='blue')

### Generalización (capas ocultas)

```r
lin<-read.csv('circle.csv',header= F)
a<-as.matrix(lin[,-3])
b<-as.matrix(lin[,3])
backprop_mlp(a,b, h=5, epochs= 500, eta = 0.1)  
# h: hidden neurons
```

Generalizar la función `backprop_mlp` anterior para que contemple la inclusión de una capa oculta. Aplicar la función al ejemplo de la clasificación circular. 

In [0]:
#::GMG::Construyo la función de FF 
feedforward <- function(x, w1, w2) {
  z1 <- cbind(1, x) %*% w1
  h <- sigmoid(z1)
  z2 <- cbind(1, h) %*% w2
  list(output = sigmoid(z2), h = h)
}

In [ ]:
backprop <- function(y,x, epochs = 10, eta = 0.1) {
### Inicializar matrices y listas

for (j in 1:epochs) {

### Propagar hacia delante
### Actualizar pesos
### Error output
print(error)

}
### Return values
return(...)
}

Si se fija el número máximo de épocas en 1000. ¿Qué número de neuronas ocultas y qué valor de la tasa de aprendizaje (eta) es óptimo para este problema? (basta una solución aproximada).

### BONUS 1 ¿Sabrías incluir un término de inercia en el método de backpropagation?

NOTA: ver transparencia 4

### BONUS 2 Generalizar a un número arbitrario de capas

Recomendación: definir los pesos como una lista de matrices, una para cada capa